### Extract radiomic features by pyradiomics.

-Feature extractor setting: see feature_extract_setting.yaml file.


**For BraTS2021 dataset**
- MRI sequences: "t1", "t1ce", "t2", "flair";
- Segmentation mask values: 
   - NCR — label 1: the necrotic tumor core;
   - ED — label 2: the peritumoral edematous/invaded tissue;
   - ET — label 4: the GD-enhancing tumor;
   
 - The sub-regions considered for evaluation are:
    - the "enhancing tumor" (ET), 
    - the "tumor core" (TC): the TC entails the ET, as well as the necrotic (NCR) parts of the tumor. 
    - the "whole tumor" (WT):The WT describes the complete extent of the disease, as it entails the TC and the peritumoral edematous/invaded tissue (ED)  
    

In [ ]:
import os
import six
import pandas as pd
import nibabel as nib
import numpy as np
import SimpleITK as sitk
import radiomics.featureextractor as featureextractor

import sys
sys.path.append("../")
from utils.myUtils import traversalDir_FirstDir, mkdir
from mySettings import get_basic_settings, get_feature_extract_setting_dict

In [ ]:
"""
Feature extractor
"""
def get_feature_extractor(log=True):
    
    basic_settings=get_basic_settings()
    extractor_setting_file=basic_settings["extractor_setting_file"]
    
    if extractor_setting_file is None:
        extractor=featureextractor.RadiomicsFeatureExtractor()
        print('Use default feature extractor!')       
    else:
        extractor=featureextractor.RadiomicsFeatureExtractor(extractor_setting_file)
      
    
    # Set binCount according to the setting
    binCount=basic_settings['binCount'].split('_')[1]
    extractor.settings['binCount']=int(binCount)

    #print the final settings of the feature extractor
    if log:
        print("extractor_setting_file path=\n ",extractor_setting_file)
        print('-----------  settings of the feature extractor---------------------')
        print('Extraction paramters:\n\t', extractor.settings)
        print("Enabled filters:\n\t", extractor.enabledImagetypes)
        print("Enabled features:\n\t", extractor.enabledFeatures)
        
    return extractor

In [ ]:
"""
Extract radiomics features from a .nii image.
"""
def extract_features_from_nii(extractor, image_path, mask_path, label):
    
    if isinstance(label, list):
        nii_image=sitk.ReadImage(image_path)
        
        orig_mask_image = sitk.ReadImage(mask_path)
        orig_mask_array = sitk.GetArrayFromImage(orig_mask_image)
        mask_array=np.zeros(orig_mask_array.shape)
        for mask_label in label:
            mask_array[orig_mask_array==mask_label]=1
        mask_image = sitk.GetImageFromArray(mask_array)
        mask_image.CopyInformation(orig_mask_image)
        
        features=extractor.execute(nii_image, mask_image,  label=1)
    else:
        features=extractor.execute(image_path, mask_path,  label=label)
        
    
    #save the feature information in a dictionary
    features_dict={}
    for key, value in six.iteritems(features):
        features_dict[key]=[value]
   
    features=pd.DataFrame(features_dict)
    return features

"""
Filter feature columns;
"""
def get_feature_columns(columns, feature_filters):
    filtered_columns=[]
    for column in columns:
        if any([column.split("_")[1]==f for f in feature_filters]):
            filtered_columns.append(column)
    
#     print("Extract {} features for each ROI of one MRI sequence!".format(len(filtered_columns)))
#     for feature in filtered_columns:
#         print("\n -{}".format(feature))
    
    return filtered_columns

"""
Extract feature for each patient, which has mutiple modalities and multiple subregion labels.
"""
def extract_features_for_one_patient(extractor, image_base_path, segmentation_base_path, patient_id,
                                     modality_list, label_dict,feature_filters):
    
    # mask file path and the label values.
    mask_path=os.path.join(segmentation_base_path, patient_id+".nii.gz")
    mask_data= nib.load(mask_path).get_fdata()
    mask_labels=np.unique(mask_data)
    assert set([int(label) for label in mask_labels]).issubset(set([0]+label_dict["wholeTumor"]))

    
    patient_base_path=os.path.join(image_base_path, patient_id)
    patient_features=None
    for sequence in modality_list:
        image_path=os.path.join(patient_base_path, patient_id+"_"+sequence+".nii.gz")
        for label_name, label in label_dict.items():
            if (label in mask_labels) or isinstance(label, list):
                features=extract_features_from_nii(extractor, image_path, mask_path, label)
                filter_columns=get_feature_columns(features.columns, feature_filters)
                features=features[filter_columns]

                # rename the columns by adding the MRI sequence names
                rename_column_dict={column: sequence+"_"+label_name+"_"+column for column in features.columns}
                features=features.rename(columns=rename_column_dict)

                if isinstance(patient_features, pd.DataFrame):
                    patient_features=pd.concat([patient_features, features], axis=1, join='outer')
                else:
                    patient_features=features

    # insert the patient name.
    patient_features.insert(0, "patient_id", patient_id)
#    print("\n===================== \nFor patient {}, features.shape={}.".format(patient_id, patient_features.shape))
#     for feature in patient_features.columns:
#         print("- {}".format(feature))
        
    return patient_features


"""
Extract features for all the patients!
"""
def main_extract_features(feature_extract_setting_dict):
    #feature extraction settings
    image_base_path=feature_extract_setting_dict["image_folder"]
    segmentation_base_path=feature_extract_setting_dict["segmentation_folder"]
    modality_list=feature_extract_setting_dict["modality_list"]
    label_dict=feature_extract_setting_dict["label_dict"]
    save_excel_path=feature_extract_setting_dict["save_excel_path"]
    feature_filters=get_basic_settings()["feature_filter"]
    
    base_results_path=os.path.dirname(save_excel_path)
    mkdir(base_results_path)
    
    # feature extractor.
    extractor=get_feature_extractor()
    
    patient_list=traversalDir_FirstDir(image_base_path)
    num_patients=len(patient_list)
    All_features=None
    i=0
    for patient_id in patient_list:
        i=i+1
        print("- {}/{}: {}".format(i, num_patients, patient_id))
        try:
            patient_features=extract_features_for_one_patient(extractor, image_base_path, segmentation_base_path, patient_id,
                                                             modality_list, label_dict, feature_filters)
        
            if isinstance(All_features, pd.DataFrame):
                All_features=pd.concat([All_features, patient_features], axis=0)
            else:
                All_features=patient_features
                
        except Exception as error:
            error_str="Caution!!! - {}/{}: {}, error: {}.".format(i, num_patients, patient_id, error)
            print(error_str)
            file = r'./error_extract_features.txt'
            with open(file, 'a+') as f:
                f.write(error_str)
     
    # set the patient id as the index
    All_features.set_index("patient_id", drop=True, inplace=True)
              
    ## save the data
    excel_writer = pd.ExcelWriter(save_excel_path)
    All_features.to_excel(excel_writer)
    excel_writer.save()
        
    return All_features

### Main

In [ ]:
"""
Main: call function and execute feature extraction.
"""

feature_extract_setting_dict=get_feature_extract_setting_dict()

for setting_name, feature_extract_settings in feature_extract_setting_dict.items():
    print("\n\n=========== Extract features for {}. ==========".format(setting_name))
    print("\n-image_folder={}, \n-segmentation_folder={}.".format(feature_extract_settings["image_folder"], feature_extract_settings["segmentation_folder"]))
    All_features=main_extract_features(feature_extract_settings)
    print("Shape of the extracted features:{} for {}.".format(All_features.shape, setting_name))
    